# AVI Graph
1. Objetivo
2. Problemas a serem resolvidos por meio desse grafo
3. Materiais 
4. Métodos

## Objetivos
O objetivo principal deste projeto é construir um grafo por meio das conversas entre usuário e AVI afim de aprender a estrutura dessas conversas, predizer os possíveis perguntas do usuário considerando a primeira pergunta, sugerir respostas alternativas e achar os fluxos mais úteis em um atendimento.

## Os problemas que pretendem-se resolver

Nessa seção vamos apresentar três problemas que pretendemos resolver.

### Primeiro caso:
Sabemos que por falta de conhecimento do domínio/produtos do banco, alguns usuários não sabem a melhor forma de explicar o problema para o atendente. Por isso, acontecem algumas conversas redundantes até o AVI ou atendente conseguir achar o problema e resolvé-lo. No caso do AVI, achar o problema verdadeiro pode demorar. Gostaríamos de utilizar os dados das perguntas semelhantes e achar a duração mais curta que consegue-se achar o problema e oferecer uma solução.

<img src="Img/Img01.png"/>




### Segundo caso:

Tendo a primeira pergunta, gostaríamos de predizer as próximas perguntas do usuário e mostrar perguntas/soluções subsequentes a fim de <b><u>facilitar</u></b> o processo ou <b><u>adiantar</u></b> a solução para o usuário.

<img src="Img/Img02.png"/>

### Terceiro caso:

Sugerir opções relevantes pode ajuar o cliuente para conhecer melhor os produtos do banco.

<img src="Img/Img03.png"/>

### Materias:

Pedem-se usar duas bases de dados: 
   1. Base das conversas do usuário com o AVI; 
   2. Base das conversas do usuário com especialista;

## Métodos

Para resolver os casos acima, usamos uma abordagem voltada à teroria de grafos. Essa solução por si consegue resolver todos os problemas acima. No entanto, no futuro algoritmos de aprendizagem de máquina tais como graph-convolutional neural networks, graph generative adversarial networks podem ser utilizadas para aprender a estrutura desse grafo e produzir novas soluções para diversos problemas interessantes.

### Proposta

Nessa seção vamos apresentar a solução sugerida. Essa solução é baseada em seguintes étapas:

1. Construa uma estrutura inicial das conversas em forma de árvore;
2. Cria relações da forma da cadeia de Markov: 

$$ P(intenção1, intenção2,... | pergunta1, pergunta 2,...)... = P(intenção1|pergunta1)P(intenção2|pergunta2)... $$

3. Adicionando novas arestas chamadas <b>"dummy edges"</b>, crie relações entre os vertíces relacionados às perguntas ( cada árvore representa uma conversa ) no espaço semântico;
4. Reduza o grafo eleminando os vértices que semanticamente são similares e tem intenções iguais
5. Calcula $ n $ matrizes das distâncias de <b> passeio aleatório </b> $ RW $ de tamanho $ l = 0,1,2,....n $ entre cada um dos vértices onde $ n = max (l) $;
6. Use as distâncias calculadas para calcular o caminho mais curto entre todos os vérticis relacionados às perguntas.

A seguir vamos explicar cada um dessas etapas.



### 1. Construir uma estrutura inicial das conversas em forma de árvore

Vamos começar com um exemplo.

<img src="Img/Img04.png">

<img src="Img/Img05.png">

### 2. Criar relações de forma da cadeia de Markov entre perguntas e as intenções

<img src="Img/Img06.png">

A estrutura das conversas mostra que cada pergunta pode ou não depender da intenção da pergunta anterior. Uma vez que por enquanto o AVI não considera o contexto (as conversas anteriores) a última pergunta é suficiente para predizer a intenção. Assim, a cadeia de Markov define a estrutura probabilística do problema da melhor forma.

In [28]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
import numpy as np
import os
from pyemd import emd
from gensim.similarities import WmdSimilarity
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

#!pip3 install pyemd

In [25]:
chat = pd.read_csv("conversas.csv" , error_bad_lines=False,  encoding="ISO-8859-1", sep = ';')

session_id=chat["session_id"]
interaction_id=chat["interaction_id"]
user_message=chat["user_message"]
intention_name=chat["intention_name"]
system_reply=chat["system_reply"]

conversations = pd.DataFrame()
conversations["sessionId"] = session_id
conversations["interaction_id"] = interaction_id
conversations["user_message"] = user_message
conversations["intention_name"] = intention_name
conversations["system_reply"] = system_reply


conversations = conversations.dropna(how="any")

conversations = conversations.drop_duplicates(['interaction_id'], keep='last')

## Sorting the dataframe based on Session Id
#conversations.sort_values(['sessionId'], ascending=[True])
conversations = conversations.sort_values(['sessionId'], ascending=[True]).drop_duplicates(['interaction_id'], keep='last')

## Deleting rows related to "Boas vindas"
conversations = conversations[conversations["intention_name"] != "boas vindas"]


### joining the conversations related to the same session
sessions = []
previousSessionId=0
session = []
loopCounter = 0
for index, item in conversations.iterrows():
    if loopCounter == 0:
        previousSessionId = item["sessionId"] 
    if previousSessionId == item["sessionId"]:
        session.append(item["sessionId"])
        #print("appended")
    else:
        sessions.append(session)
        session = []
    previousSessionId = item["sessionId"]
    loopCounter += 1

conversations

/home/vfcarida/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (16,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 3. Dummy edges
Nessa etapa existem várias árvores que representam diversas conversas. Uma vez que algumas perguntas em conversas distintas tem intenções iguais, existem arestas que conectam essas árvores. Todavia, essas relações são somente estruturais. Para resolver os primeiro e terceiro problemas, é necessário considerar relações semânticas entre as perguntas.

<b>Mas como criar essas relações entre dados estruturais e o espaço semantico?</b>

Para fazer isso, propomos adicionar novas arestas chamadas <b>"dummy edges"</b>. Essas arestas não existem no grafo original mas ajudam a levar os atributis (attribute graph) dos vértices a consideração. Nessa etapa propõe-se o seguinte algoritmo:

1. Atribua pesos iguais para todas as arestas $ W_{i,j}  = 1$
2. Ache a distância semântica $ d_{i,j} $ entre todos os vértices do grafo, $ i $ e $ j $
3. Crie arestas entre os vérticies e atribua pesos de forma $ W_{i,j} = 1 - \frac{d_{i,j}}{D} $ onde $ D $ é maior distância entre dois vértices no grafo, $ W_{i,j}$ é a probabilidade de chegar de $i$ a $j$. Nessa equação $ i $ e $ j $ não são vértices da mesma àrvore (conversa). Chamamos essas arestas de "dummy edges". Já que os "dummy edges" não fazem parte do grafo original, sempre a preferênia de escolher um caminho entre dois vérticies é o caminho do grafo original. Isso justifica a existência de 1 na equação $ W_{i,j} = 1 - \frac{d_{i,j}}{D} $.

<img src="Img/Img07.png">
   
Nesse exemplo $ d_{1,6} $ refere a distância semântica entre vértice número 1 e 6. Assim formamos $ W_{1,6} = 1 + d_{1,6} $

In [ ]:
import string
## Deleting the punctuation from conversations
sentences = []
for line in sessionText:
    line = "".join(l for l in line if l not in string.punctuation)
    sentences.append(line.lower().split())
    #sentences.append(line.lower())    


In [ ]:
## Training Word2Vec on conversations in order to get the semantic similarity for conversation words 
chatModel = Word2Vec(sentences, min_count=1)

## Testing the trained model

chatModel.wv.most_similar("depositado")

In [ ]:
#generating conversation tree using adjecency matrix


intentionsList=[]
counter = 0
for key in intentions:
     intentionsList.append(key)



nodeIndex=0
dummynodeIndex=0
nodeIndexLabel = []
# 348 : number of existing intentions
# 261783 : number of conversations
# WE USE THE FIRST 35000 SAMPLES TO LEARN


#### Creating the matrix of only 35000 conversations
previousSessionId = None
conversationLen= len(conversations[0:35000])
adjMatrix = np.zeros((conversationLen , conversationLen + len(intentions)))

for index ,chat in conversations[0:conversationLen].iterrows():
    currentId = chat["sessionId"]
    ## setting the intention in adj matrix
    intentionIndex = intentionsList.index(chat["intention_name"])
    adjMatrix[nodeIndex][conversationLen + intentionIndex -1] = 1
    
    ## setting the chats tree structures in adj matrix
    if previousSessionId == None:
        previousSessionId = chat["sessionId"]
    if previousSessionId == currentId:
        adjMatrix[nodeIndex-1][nodeIndex] = 1
        
    previousSessionId = currentId
    nodeIndex += 1

In [ ]:
## Generating conversation tree using List (more economic!!!!!) and adding DUMMY NODES
nodeIndex=0
nodeIndexLabel = []

previousSessionId = None

adjList = []
conversationLen= len(conversations[0:10000])
dummynodeIndex = conversationLen
# 2 * conversationLen was generated in order to have both semantic and structural data

for index ,chat in conversations[0:conversationLen].iterrows():
    currentId = chat["sessionId"]
    ## setting the intention in adjcency list
    intentionIndex = intentionsList.index(chat["intention_name"])
    ## WE DO NOT NEED "dummy nodes" BECAUSE IT WAS REPLACED WITH DUMMY EDGES
    ######adjList.append((nodeIndex , conversationLen + dummynodeIndex + intentionIndex))
    ##
    
    adjList.append((nodeIndex , conversationLen + intentionIndex))
    
    ## setting the dummy node related to node attribute (semantic vector)
    ######adjList.append((nodeIndex, dummynodeIndex + nodeIndex))
    ##
    adjList.append((nodeIndex, nodeIndex))
    ## setting the chats tree structures in adj matrix
    
    if previousSessionId == currentId:
        adjList.append((nodeIndex-1,nodeIndex))
    if previousSessionId == None:
        previousSessionId = chat["sessionId"]
        
    previousSessionId = currentId
    nodeIndex += 1


In [ ]:
## Calculating the semantic distance between conversations
## CAUTION!!: NO NEED TO RUN THIS CODE! THIS PART HAS ALREADY BEEN DONE AND SAVED IN A TEXT FILE. Running this file
## will take 2 dayw! So, use the "chatSemanticSimilarity.txt" file!
## Calculating the semantic distance between conversations
import threading
usersMessagesDistances = []

def getDistance(counter,i,str1,str2):
    distance = chatModel.wmdistance(str1,str2)
    usersMessagesDistances.append((counter, i ,distance))
    if len(usersMessagesDistances)%10000000==0:
        with open("chatSemanticSimilarities.txt","w") as writer:
            for item in usersMessagesDistances:
                writer.write(str(item))
                writer.write("\n")

counter = 0
for index ,chat in conversations[0:conversationLen].iterrows():
    for i in range(counter, conversationLen):
        arg = (counter, i , chat["user_message"].lower(),list(conversations['user_message'])[i].lower())
        threading.Thread(target = getDistance , args = arg).start()
    counter+=1
    #if counter%100==0:
    #print(counter)

In [ ]:
## To read the the message disatnces from file
usersMessagesDistances = []
with open('chatSemanticSimilarities.txt') as f:
    for item in f:
        usersMessagesDistances = item.split(";")
        #print(usersMessagesDistances[0])

### 4. Redução do grafo

Neste passo o objetivo é reduzir o tamanho do grafo afim de ter uma estrutura numérica que passa maior informação tendo o menor tamanho. Escolhemos um parâmetro <b>$ α $</b> como limiar. Essa fase possui duas etapas:
1. Eliminar os vértices inúteis:

    -- Se o peso <b>$ W_{i,j} $</b> relacionado a dummy edge <b>$ E_{i,j} $</b> é menor de <b>$ α $</b> e <b>$ i $</b> e <b>$ j $</b> têm intenções iguais, elemine <b>$ j $</b> e a aresta que conecta ele à intenção e todas as arestas que conectam os vértices que são da mesma conversa que <b>$ i $</b> a <b>$ j $</b>.   


2. Reconstruir as arestas da entrada e saída desses vértices:

     -- Conecte a entrada de <b>$ j $</b> a <b>$ i $</b> sem nenhuma mudança em peso <br>
     -- conecte <b>$ i $</b> aos filhos (que não são intenções) de <b>$ j $</b> e atualize os pesos de forma: 
     $$ W_{i,k} = W_{i,j} * W_{j,k} $$
     
Como pode-se ver na figura.b, vértices 3 e 5 têm intenções iguais. Se $ d_{i,j} < α $ podemos eliminar 5 e atualizar os pesos. Caso <b>$ j $</b> a <b>$ i $</b> estejam apontando para mesmo vértice, atualizamos a aresta com peso menor. 
É importante mencionar que nesse processo somente ajustamos os pesos das dummy edges e os pessos das arestas originais permanecem 1.

<img src="Img/img08.png">


In [ ]:
### Joining similar nodes
counter = 0
singleNodeDistances = []
for distance in usersMessagesDistances:
    if(counter <= 10000):
        nodeSpec = distance.split("(")[1].split(",")
        ## If the distance is 0, it means that two questions are exactly the same
        if(nodeSpec[0]=="0"):
            singleNodeDistances.append(float(nodeSpec[2].split(")")[0]))
    counter +=1

## Como aprender esse grafo?

A ideia pricipal é aprender as distâncias de passeios aleatóries entre os vértices do grafo.

Calculamos as distâncias de passeio aleatório por meio da seguinte fórmula:

$$ d(v_{i},v_{j}) = \sum_{\tau:v_{i\hookrightarrow}v_{j}}{p_{v_{i},v_{j}}(\tau)c(1-c)^{\tau}} $$ onde $ \tau $ é o probabilidade do passeio aleatório chegar de $ v_{i} $ a $ v_{j} $, $ c $ é a probabilidade de volta para $ v_{i} $. Vamos fixar um número pequeno para probabilidade de volta. Definimos $ p(\tau) $ como:

$$ p_{v_{i},v_{j}}=\frac{W_{i,j} }{\sum W_{vizinhos}} $$


Assim criamos uma apresentação vetorial de todas as distâncias onde cada distância representa um feature do grafo. A seguir vamos criar uma apresentação vetorial de um grafo simples:

<img src="Img/Img09.png">

Antes vamos dar alguns valores às distâncias semânticas entre os vértices:

$d_{1,4} = 0.4 $ , $d_{1,5} = 9 $, $d_{2,4} = 3 $, $d_{2,5} = 11 $, $d_{3,4} = 5 $, $d_{3,5} = 0.2 $

Os pesos relacionados às arestas originais têm peso de 1. Assim:

$W_{1,1} = 0 $
$W_{1,2} = 1 $
$W_{1,3} = 0 $
$W_{2,1} = 0 $ <br>
$W_{2,2} = 0 $
$W_{2,3} = 1 $
$W_{3,1} = 0 $ <br>
$W_{3,2} = 0 $
$W_{3,3} = 0 $

e os pesos entre dummy edges são calculados como seguinte:

$W_{1,4} = 1 - \frac{d_{1,4}}{ 11 = D} = 1 - \frac{0.4}{11} = 0.96 $ <br> 
$W_{1,5} = 1 - \frac{d_{1,5}}{ 11 = D} = 1 - \frac{9}{11} = 0.19 $ <br> 
$W_{2,4} = 1 - \frac{d_{2,4}}{ 11 = D} = 1 - \frac{3}{11} = 0.73 $ <br> 
$W_{2,5} = 1 - \frac{d_{2,5}}{ 11 = D} = 1 - \frac{11}{11} = 0 $ <br> 
$W_{3,4} = 1 - \frac{d_{3,4}}{ 11 = D} = 1 - \frac{5}{11} = 0.65 $ <br> 
$W_{3,5} = 1 - \frac{d_{3,5}}{ 11 = D} = 1 - \frac{0.2}{11} = 0.98 $ <br>

Como pode-se ver, para os vértices que semanticamente são mais parecidos, a probabilidade de transição é mais alta do que os vértices que são semanaticamente mais longes.

Vamos fixar $ \tau = 2 $ e $ c = 0.0002 $. Assim vamos calcular a distância baseado em passeios alearórios de tamanho 2. Mostramos essas distâncias com $ d_{rw}(v_{i}, v_{j})$



$ d_{rw}(v_{1}, v_{2}) =  \sum_{\tau=1}^{2} c(1-c) P_{v_{1}, v_{2}} = (0.0002 \times 0.998 \times  \frac{1}{1+1+(W_{1,4}=0.96)}) + (0.0002) \times 0.998 \times (p_{1,4}=\frac{0.96}{1+1+(W_{1,4}=0.96)}) \times (p_{2,4}=\frac{0.73}{1+1+(W_{1,4}=0.96 + W_{2,4}=0.73 + W_{2,4}=0.65)}) = 0.00001 $

Assim, calculamos as distâncias para todos os vértices e formamos uma respresentação vetorial do grafo.

<img src="Img/Img10.png">